# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 

As of today, the easiest way to experiment with Ebiose is to use the OpenAI API. To do so, all you have to do is to set your OpenAI API key via an .env file or by replacing `"your-open-api-key"` in the following code block:

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

> 💡 Recall that you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so or execute the following:


In [2]:
import sys
sys.path.append("./../")


## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [3]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [4]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [5]:
import random
random.seed(7)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [6]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id="azure-gpt-4o-mini"
)

# Running a forge cycle

Once the forge is instantitated, we can start generating agents by running a **forge cycle**. 

We must first define a list of available LLMs. Here, we will use GPT 4o mini only. 

In Ebiose, all calls to LLMs (and, in a next future, for any other type of call that costs) are managed by the `ComputeIntensiveBatchProcessor` static class. This class must be initialized with the list of available models, as follows:

In [ ]:

from ebiose.compute_intensive_batch_processor.compute_intensive_batch_processor import (
    ComputeIntensiveBatchProcessor,
)
ComputeIntensiveBatchProcessor.initialize()

The forge cycle is based on an evolutionary algorithm that requires two types of specialized agents: 
- **architect agents** which are agents that generate other agents from scratch;
- **genetic operator agents** which generate agents from one or several other existing agents.

For now, hand-made architect and crossover agents are provided in the `GraphUtils` class. We can load them as follows:

In [10]:
from ebiose.core.engines.graph_engine.utils import GraphUtils

architect_agent = GraphUtils.get_architect_agent(model_endpoint_id="azure-gpt-4o-mini")
crossover_agent = GraphUtils.get_crossover_agent(model_endpoint_id="azure-gpt-4o-mini")

These agents are required to instantiate an Ebiose ecosystem:

In [11]:
from ebiose.core.ecosystem import Ecosystem

ecosystem = Ecosystem(
    initial_architect_agents=[architect_agent],
    initial_genetic_operator_agents=[crossover_agent],
)

The forge cycle can now be launched by providing the created ecosystem, a budget in dollars (the forge cycle will end once this budget is exhausted) and, optionally, a path in which created agents and fitness will be saved accross generations. Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [ ]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCylceConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)

# budget for the forge cycle, in dollars
BUDGET = 0.01

cycle_config = EvoForgingCylceConfig(
    budget=BUDGET,
    n_agents_in_population=2,
    n_selected_agents_from_ecosystem=0,
    replacement_ratio=0.5,
    save_path=SAVE_PATH
)

final_agents, final_fitness = asyncio.run(forge.run_new_cycle(ecosystem, cycle_config=cycle_config))


Starting a new cycle for forge Basic forge
****** Initializing agents population ******
Creating 2 new agents with architect agents...
100%|██████████| 2/2 [00:36<00:00, 18.10s/it]
Agent initialization cost: 0.007469220000000001
Population initialized with 2 agents
Initialization of 2 agents took 0:00:36.235720
Budget left after initialization: 0.012530779999999998 $
****** Running generation 0 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 1485.76it/s]
Agent agent-f337b468-86d6-4612-9540-2da9f3de9ca7 fitness: 0.7798296305842437, cost: 0.0
Agent agent-cb344596-6e33-49f4-9f08-476ced3dbcea fitness: 0.08185501079576984, cost: 0.0
Evaluation took 0:00:00.010860 for a total cost of 0.0 $


# Agent ID: agent-f337b468-86d6-4612-9540-2da9f3de9ca7
## Fitness: 0.7798296305842437
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(Math Problem Solver)
	Math_Problem_Solver(Math Problem Solver) --> Equation_Formulator(Equation Formulator)
	Equation_Formulator(Equation Formulator) --> Problem_Solver(Problem Solver)
	Problem_Solver(Problem Solver) --> Solution_Verifier(Solution Verifier)
	Solution_Verifier(Solution Verifier) --> Alternative_Methods_Explorer(Alternative Methods Explorer)
	Alternative_Methods_Explorer(Alternative Methods Explorer) --> Contextual_Reflection(Contextual Reflection)
	Contextual_Reflection(Contextual Reflection) --> Feedback_Generator(Feedback Generator)
	Feedback_Generator(Feedback Generator) --> Summary_Creator(Summary Creator)
	Summary_Creator(Summary Creator) --> Knowledge_Integrator(Knowledge Integrator)
	Knowledge_Integrator(Knowledge Integrator) --> Final_Output_Formatter(Final Output Formatter)
	Final_Output_Formatter(Final Output Formatter) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
Welcome to the collaborative math problem-solving network. This graph consists of multiple nodes, each representing a distinct stage in the problem-solving process.
##### Math Problem Solver
Analyze the following math word problem and identify the key components (numbers, operations, etc.): {problem_statement}
##### Equation Formulator
Based on the analysis from the Math Problem Solver, your role is to formulate an equation that accurately represents the problem identified. Ensure that the equation includes all relevant variables and constants derived from the previous node’s output.
##### Problem Solver
With the equation formulated, your task is to solve it. Apply appropriate mathematical techniques to find the solution, including algebraic manipulation or numerical methods as necessary.
##### Solution Verifier
Your responsibility is to evaluate the solution derived by the Problem Solver. Check for accuracy by substituting the solution back into the original equation.
##### Alternative Methods Explorer
Consider alternative methods that could be employed to solve the original problem. Discuss different strategies that could yield the same result.
##### Contextual Reflection
Reflect on the entire problem-solving process, including the strategies used, the effectiveness of the solutions obtained, and the communication between nodes.
##### Feedback Generator
Generate constructive feedback based on the evaluations provided by both the Solution Verifier and the Contextual Reflection nodes.
##### Summary Creator
Create a comprehensive summary of the entire problem-solving process. Include the original math word problem, the key findings from each node, the solution derived, and the feedback gathered.
##### Knowledge Integrator
Integrate knowledge and techniques from previous nodes to suggest additional resources or strategies that could assist in similar problem-solving scenarios.
##### Final Output Formatter
Your role is to format the final output of the problem-solving process for presentation.

# Agent ID: agent-cb344596-6e33-49f4-9f08-476ced3dbcea
## Fitness: 0.08185501079576984
```mermaid 
graph LR
	Start_Node[StartNode] --> Problemunderstanding(ProblemUnderstanding)
	Problemunderstanding(ProblemUnderstanding) --> Problemsolving(ProblemSolving)
	Problemsolving(ProblemSolving) --> End_Node[EndNode]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network designed to solve math word problems. Each of you plays a specific role in this problem-solving process. The flow of interaction begins with understanding the problem, followed by calculating the solution, and finally concluding with the output. Your task is to communicate effectively with other nodes, ensure clarity of thought, and utilize chain-of-thought reasoning, self-reflection, and evaluative feedback mechanisms in your responses.
##### ProblemUnderstanding
You are tasked with understanding a math word problem. Carefully analyze the text of the problem and identify the key variables, operations, and relationships involved. Break down the problem into its essential components and articulate the understanding in a clear and concise manner. If there are ambiguities or multiple interpretations, reflect on these and provide a brief analysis of each. Finally, convey your understanding to the next node, ensuring all necessary details are included for the subsequent steps.
##### ProblemSolving
You have received an interpretation of a math word problem from the previous node. Based on this understanding, apply appropriate mathematical techniques to derive a solution. Clearly outline the steps taken in your calculations, including any assumptions made during the process. If applicable, reflect on the accuracy of your approach and consider alternative methods that could lead to the same or a different solution. Present your final answer in a clear format, ensuring that all relevant information is communicated to the EndNode.



Saving current state to ../data/2025-03-04_18-51-10/generation=0
Starting crossover and mutation...
Initializing structured output agent for model with fields {'edges': FieldInfo(annotation=list[Edge], required=False, default_factory=list, description='list of edges in the graph'), 'nodes': FieldInfo(annotation=list[Union[StartNode, EndNode, LLMNode, PydanticValidatorNode, RegexRoutingNode]], required=False, default_factory=list, description='list of nodes in the graph'), 'description': FieldInfo(annotation=str, required=False, default=''), 'shared_context_prompt': FieldInfo(annotation=str, required=True, metadata=[MinLen(min_length=1)])} (2)
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:27.616214 for a total cost of 0.0028053300000000004 $
Generation 0 completed in 0:00:27.669116 with a total cost of 0.0028053300000000004 $
Budget left after first generation: 0.00972545 $
****** Running generation 1 ******
Evaluating current population of 2 agents...
100%|████████

# Agent ID: agent-758a6124-56c7-476c-a4dd-e4b411a1f525
## Fitness: 0.7890941714903549
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(Math Problem Solver)
	Math_Problem_Solver(Math Problem Solver) --> Equation_Formulator(Equation Formulator)
	Equation_Formulator(Equation Formulator) --> Problem_Solver(Problem Solver)
	Problem_Solver(Problem Solver) --> Solution_Verifier(Solution Verifier)
	Solution_Verifier(Solution Verifier) --> Alternative_Methods_Explorer(Alternative Methods Explorer)
	Alternative_Methods_Explorer(Alternative Methods Explorer) --> Contextual_Reflection(Contextual Reflection)
	Contextual_Reflection(Contextual Reflection) --> Feedback_Generator(Feedback Generator)
	Feedback_Generator(Feedback Generator) --> Summary_Creator(Summary Creator)
	Summary_Creator(Summary Creator) --> Knowledge_Integrator(Knowledge Integrator)
	Knowledge_Integrator(Knowledge Integrator) --> Final_Output_Formatter(Final Output Formatter)
	Final_Output_Formatter(Final Output Formatter) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
Welcome to the collaborative math problem-solving network. This graph consists of multiple nodes, each representing a distinct stage in the problem-solving process.
##### Math Problem Solver
Analyze the following math word problem and identify the key components (numbers, operations, etc.): {problem_statement}
##### Equation Formulator
Based on the analysis from the Math Problem Solver, your role is to formulate an equation that accurately represents the problem identified. Include alternative formulations if applicable.
##### Problem Solver
With the equation formulated, your task is to solve it. Apply appropriate mathematical techniques, including algebraic manipulation, numerical methods, or graphical representation as necessary.
##### Solution Verifier
Your responsibility is to evaluate the solution derived by the Problem Solver. Check for accuracy by substituting the solution back into the original equation and validate with alternative methods.
##### Alternative Methods Explorer
Consider alternative methods that could be employed to solve the original problem. Discuss different strategies that could yield the same result and their potential advantages.
##### Contextual Reflection
Reflect on the entire problem-solving process, including the strategies used, the effectiveness of the solutions obtained, and the communication between nodes. Suggest improvements for future problem-solving scenarios.
##### Feedback Generator
Generate constructive feedback based on the evaluations provided by both the Solution Verifier and the Contextual Reflection nodes. Identify areas for improvement.
##### Summary Creator
Create a comprehensive summary of the entire problem-solving process. Include the original math word problem, the key findings from each node, the solution derived, and the feedback gathered.
##### Knowledge Integrator
Integrate knowledge and techniques from previous nodes to suggest additional resources or strategies that could assist in similar problem-solving scenarios.
##### Final Output Formatter
Your role is to format the final output of the problem-solving process for presentation.

# Agent ID: agent-f337b468-86d6-4612-9540-2da9f3de9ca7
## Fitness: 0.5580757526533747
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(Math Problem Solver)
	Math_Problem_Solver(Math Problem Solver) --> Equation_Formulator(Equation Formulator)
	Equation_Formulator(Equation Formulator) --> Problem_Solver(Problem Solver)
	Problem_Solver(Problem Solver) --> Solution_Verifier(Solution Verifier)
	Solution_Verifier(Solution Verifier) --> Alternative_Methods_Explorer(Alternative Methods Explorer)
	Alternative_Methods_Explorer(Alternative Methods Explorer) --> Contextual_Reflection(Contextual Reflection)
	Contextual_Reflection(Contextual Reflection) --> Feedback_Generator(Feedback Generator)
	Feedback_Generator(Feedback Generator) --> Summary_Creator(Summary Creator)
	Summary_Creator(Summary Creator) --> Knowledge_Integrator(Knowledge Integrator)
	Knowledge_Integrator(Knowledge Integrator) --> Final_Output_Formatter(Final Output Formatter)
	Final_Output_Formatter(Final Output Formatter) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
Welcome to the collaborative math problem-solving network. This graph consists of multiple nodes, each representing a distinct stage in the problem-solving process.
##### Math Problem Solver
Analyze the following math word problem and identify the key components (numbers, operations, etc.): {problem_statement}
##### Equation Formulator
Based on the analysis from the Math Problem Solver, your role is to formulate an equation that accurately represents the problem identified. Ensure that the equation includes all relevant variables and constants derived from the previous node’s output.
##### Problem Solver
With the equation formulated, your task is to solve it. Apply appropriate mathematical techniques to find the solution, including algebraic manipulation or numerical methods as necessary.
##### Solution Verifier
Your responsibility is to evaluate the solution derived by the Problem Solver. Check for accuracy by substituting the solution back into the original equation.
##### Alternative Methods Explorer
Consider alternative methods that could be employed to solve the original problem. Discuss different strategies that could yield the same result.
##### Contextual Reflection
Reflect on the entire problem-solving process, including the strategies used, the effectiveness of the solutions obtained, and the communication between nodes.
##### Feedback Generator
Generate constructive feedback based on the evaluations provided by both the Solution Verifier and the Contextual Reflection nodes.
##### Summary Creator
Create a comprehensive summary of the entire problem-solving process. Include the original math word problem, the key findings from each node, the solution derived, and the feedback gathered.
##### Knowledge Integrator
Integrate knowledge and techniques from previous nodes to suggest additional resources or strategies that could assist in similar problem-solving scenarios.
##### Final Output Formatter
Your role is to format the final output of the problem-solving process for presentation.



Saving current state to ../data/2025-03-04_18-51-10/generation=1
Starting crossover and mutation...
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:28.646283 for a total cost of 0.002836845 $
Generation 1 completed in 0:00:28.668077 with a total cost of 0.002836845 $
Budget left after new generation: 0.006888604999999999 $
****** Running generation 2 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 2840.71it/s]
Agent agent-758a6124-56c7-476c-a4dd-e4b411a1f525 fitness: 0.4456407672509217, cost: 0.0
Agent agent-6802069f-577f-4d2e-b539-b3de3f59d36e fitness: 0.7166277943983036, cost: 0.0
Evaluation took 0:00:00.003577 for a total cost of 0.0 $


# Agent ID: agent-6802069f-577f-4d2e-b539-b3de3f59d36e
## Fitness: 0.7166277943983036
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(Math Problem Solver)
	Math_Problem_Solver(Math Problem Solver) --> Equation_Formulator(Equation Formulator)
	Equation_Formulator(Equation Formulator) --> Problem_Solver(Problem Solver)
	Problem_Solver(Problem Solver) --> Solution_Verifier(Solution Verifier)
	Solution_Verifier(Solution Verifier) --> Alternative_Methods_Explorer(Alternative Methods Explorer)
	Alternative_Methods_Explorer(Alternative Methods Explorer) --> Contextual_Reflection(Contextual Reflection)
	Contextual_Reflection(Contextual Reflection) --> Feedback_Generator(Feedback Generator)
	Feedback_Generator(Feedback Generator) --> Summary_Creator(Summary Creator)
	Summary_Creator(Summary Creator) --> Knowledge_Integrator(Knowledge Integrator)
	Knowledge_Integrator(Knowledge Integrator) --> Final_Output_Formatter(Final Output Formatter)
	Final_Output_Formatter(Final Output Formatter) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
Welcome to the collaborative math problem-solving network. This graph consists of multiple nodes, each representing a distinct stage in the problem-solving process.
##### Math Problem Solver
Analyze the following math word problem and identify the key components (numbers, operations, etc.): {problem_statement}
##### Equation Formulator
Based on the analysis from the Math Problem Solver, your role is to formulate an equation that accurately represents the problem identified. Include alternative formulations if applicable.
##### Problem Solver
With the equation formulated, your task is to solve it. Apply appropriate mathematical techniques, including algebraic manipulation, numerical methods, or graphical representation as necessary.
##### Solution Verifier
Your responsibility is to evaluate the solution derived by the Problem Solver. Check for accuracy by substituting the solution back into the original equation and validate with alternative methods.
##### Alternative Methods Explorer
Consider alternative methods that could be employed to solve the original problem. Discuss different strategies that could yield the same result and their potential advantages.
##### Contextual Reflection
Reflect on the entire problem-solving process, including the strategies used, the effectiveness of the solutions obtained, and the communication between nodes. Suggest improvements for future problem-solving scenarios.
##### Feedback Generator
Generate constructive feedback based on the evaluations provided by both the Solution Verifier and the Contextual Reflection nodes. Identify areas for improvement.
##### Summary Creator
Create a comprehensive summary of the entire problem-solving process. Include the original math word problem, the key findings from each node, the solution derived, and the feedback gathered.
##### Knowledge Integrator
Integrate knowledge and techniques from previous nodes to suggest additional resources or strategies that could assist in similar problem-solving scenarios.
##### Final Output Formatter
Your role is to format the final output of the problem-solving process for presentation.

# Agent ID: agent-758a6124-56c7-476c-a4dd-e4b411a1f525
## Fitness: 0.4456407672509217
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(Math Problem Solver)
	Math_Problem_Solver(Math Problem Solver) --> Equation_Formulator(Equation Formulator)
	Equation_Formulator(Equation Formulator) --> Problem_Solver(Problem Solver)
	Problem_Solver(Problem Solver) --> Solution_Verifier(Solution Verifier)
	Solution_Verifier(Solution Verifier) --> Alternative_Methods_Explorer(Alternative Methods Explorer)
	Alternative_Methods_Explorer(Alternative Methods Explorer) --> Contextual_Reflection(Contextual Reflection)
	Contextual_Reflection(Contextual Reflection) --> Feedback_Generator(Feedback Generator)
	Feedback_Generator(Feedback Generator) --> Summary_Creator(Summary Creator)
	Summary_Creator(Summary Creator) --> Knowledge_Integrator(Knowledge Integrator)
	Knowledge_Integrator(Knowledge Integrator) --> Final_Output_Formatter(Final Output Formatter)
	Final_Output_Formatter(Final Output Formatter) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
Welcome to the collaborative math problem-solving network. This graph consists of multiple nodes, each representing a distinct stage in the problem-solving process.
##### Math Problem Solver
Analyze the following math word problem and identify the key components (numbers, operations, etc.): {problem_statement}
##### Equation Formulator
Based on the analysis from the Math Problem Solver, your role is to formulate an equation that accurately represents the problem identified. Include alternative formulations if applicable.
##### Problem Solver
With the equation formulated, your task is to solve it. Apply appropriate mathematical techniques, including algebraic manipulation, numerical methods, or graphical representation as necessary.
##### Solution Verifier
Your responsibility is to evaluate the solution derived by the Problem Solver. Check for accuracy by substituting the solution back into the original equation and validate with alternative methods.
##### Alternative Methods Explorer
Consider alternative methods that could be employed to solve the original problem. Discuss different strategies that could yield the same result and their potential advantages.
##### Contextual Reflection
Reflect on the entire problem-solving process, including the strategies used, the effectiveness of the solutions obtained, and the communication between nodes. Suggest improvements for future problem-solving scenarios.
##### Feedback Generator
Generate constructive feedback based on the evaluations provided by both the Solution Verifier and the Contextual Reflection nodes. Identify areas for improvement.
##### Summary Creator
Create a comprehensive summary of the entire problem-solving process. Include the original math word problem, the key findings from each node, the solution derived, and the feedback gathered.
##### Knowledge Integrator
Integrate knowledge and techniques from previous nodes to suggest additional resources or strategies that could assist in similar problem-solving scenarios.
##### Final Output Formatter
Your role is to format the final output of the problem-solving process for presentation.



Saving current state to ../data/2025-03-04_18-51-10/generation=2
Starting crossover and mutation...
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:21.560459 for a total cost of 0.0028281 $
Generation 2 completed in 0:00:21.571426 with a total cost of 0.0028281 $
Budget left after new generation: 0.004060504999999999 $
****** Running generation 3 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 3675.99it/s]
Agent agent-6802069f-577f-4d2e-b539-b3de3f59d36e fitness: 0.44918740094933096, cost: 0.0
Agent agent-d384aba8-f8ab-4421-bdf0-2df377fb326a fitness: 0.5494399091440374, cost: 0.0
Evaluation took 0:00:00.003385 for a total cost of 0.0 $


# Agent ID: agent-d384aba8-f8ab-4421-bdf0-2df377fb326a
## Fitness: 0.5494399091440374
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(Math Problem Solver)
	Math_Problem_Solver(Math Problem Solver) --> Equation_Formulator(Equation Formulator)
	Equation_Formulator(Equation Formulator) --> Problem_Solver(Enhanced Problem Solver)
	Problem_Solver(Enhanced Problem Solver) --> Solution_Verifier(Solution Verifier)
	Solution_Verifier(Solution Verifier) --> Alternative_Methods_Explorer(Alternative Methods Explorer)
	Alternative_Methods_Explorer(Alternative Methods Explorer) --> Contextual_Reflection(Contextual Reflection)
	Contextual_Reflection(Contextual Reflection) --> Feedback_Generator(Feedback Generator)
	Feedback_Generator(Feedback Generator) --> Summary_Creator(Summary Creator)
	Summary_Creator(Summary Creator) --> Knowledge_Integrator(Knowledge Integrator)
	Knowledge_Integrator(Knowledge Integrator) --> Final_Output_Formatter(Final Output Formatter)
	Final_Output_Formatter(Final Output Formatter) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
Welcome to the collaborative math problem-solving network. This graph consists of multiple nodes, each representing a distinct stage in the problem-solving process.
##### Math Problem Solver
Analyze the following math word problem and identify the key components (numbers, operations, etc.): {problem_statement}
##### Equation Formulator
Based on the analysis from the Math Problem Solver, your role is to formulate an equation that accurately represents the problem identified. Include alternative formulations if applicable.
##### Enhanced Problem Solver
With the equation formulated, your task is to solve it. Apply appropriate mathematical techniques, including algebraic manipulation, numerical methods, or graphical representation as necessary. Provide detailed steps.
##### Solution Verifier
Your responsibility is to evaluate the solution derived by the Problem Solver. Check for accuracy by substituting the solution back into the original equation and validate with alternative methods.
##### Alternative Methods Explorer
Consider alternative methods that could be employed to solve the original problem. Discuss different strategies that could yield the same result and their potential advantages.
##### Contextual Reflection
Reflect on the entire problem-solving process, including the strategies used, the effectiveness of the solutions obtained, and the communication between nodes. Suggest improvements for future problem-solving scenarios.
##### Feedback Generator
Generate constructive feedback based on the evaluations provided by both the Solution Verifier and the Contextual Reflection nodes. Identify areas for improvement.
##### Summary Creator
Create a comprehensive summary of the entire problem-solving process. Include the original math word problem, the key findings from each node, the solution derived, and the feedback gathered.
##### Knowledge Integrator
Integrate knowledge and techniques from previous nodes to suggest additional resources or strategies that could assist in similar problem-solving scenarios.
##### Final Output Formatter
Your role is to format the final output of the problem-solving process for presentation.

# Agent ID: agent-6802069f-577f-4d2e-b539-b3de3f59d36e
## Fitness: 0.44918740094933096
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(Math Problem Solver)
	Math_Problem_Solver(Math Problem Solver) --> Equation_Formulator(Equation Formulator)
	Equation_Formulator(Equation Formulator) --> Problem_Solver(Problem Solver)
	Problem_Solver(Problem Solver) --> Solution_Verifier(Solution Verifier)
	Solution_Verifier(Solution Verifier) --> Alternative_Methods_Explorer(Alternative Methods Explorer)
	Alternative_Methods_Explorer(Alternative Methods Explorer) --> Contextual_Reflection(Contextual Reflection)
	Contextual_Reflection(Contextual Reflection) --> Feedback_Generator(Feedback Generator)
	Feedback_Generator(Feedback Generator) --> Summary_Creator(Summary Creator)
	Summary_Creator(Summary Creator) --> Knowledge_Integrator(Knowledge Integrator)
	Knowledge_Integrator(Knowledge Integrator) --> Final_Output_Formatter(Final Output Formatter)
	Final_Output_Formatter(Final Output Formatter) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
Welcome to the collaborative math problem-solving network. This graph consists of multiple nodes, each representing a distinct stage in the problem-solving process.
##### Math Problem Solver
Analyze the following math word problem and identify the key components (numbers, operations, etc.): {problem_statement}
##### Equation Formulator
Based on the analysis from the Math Problem Solver, your role is to formulate an equation that accurately represents the problem identified. Include alternative formulations if applicable.
##### Problem Solver
With the equation formulated, your task is to solve it. Apply appropriate mathematical techniques, including algebraic manipulation, numerical methods, or graphical representation as necessary.
##### Solution Verifier
Your responsibility is to evaluate the solution derived by the Problem Solver. Check for accuracy by substituting the solution back into the original equation and validate with alternative methods.
##### Alternative Methods Explorer
Consider alternative methods that could be employed to solve the original problem. Discuss different strategies that could yield the same result and their potential advantages.
##### Contextual Reflection
Reflect on the entire problem-solving process, including the strategies used, the effectiveness of the solutions obtained, and the communication between nodes. Suggest improvements for future problem-solving scenarios.
##### Feedback Generator
Generate constructive feedback based on the evaluations provided by both the Solution Verifier and the Contextual Reflection nodes. Identify areas for improvement.
##### Summary Creator
Create a comprehensive summary of the entire problem-solving process. Include the original math word problem, the key findings from each node, the solution derived, and the feedback gathered.
##### Knowledge Integrator
Integrate knowledge and techniques from previous nodes to suggest additional resources or strategies that could assist in similar problem-solving scenarios.
##### Final Output Formatter
Your role is to format the final output of the problem-solving process for presentation.



Saving current state to ../data/2025-03-04_18-51-10/generation=3
Starting crossover and mutation...
BudgetExceededError: Master budget limit exceeded. Limit: 0.02, New total: 0.020730105. Finishing process.
Error when calling azure-gpt-4o-mini: Master budget limit exceeded. Limit: 0.02, New total: 0.020730105
BudgetExceededError: Master budget limit exceeded. Limit: 0.02, New total: 0.022508145. Finishing process.
Error when calling azure-gpt-4o-mini: Master budget limit exceeded. Limit: 0.02, New total: 0.022508145
BudgetExceededError: Master budget limit exceeded. Limit: 0.02, New total: 0.02429196. Finishing process.
Error when calling azure-gpt-4o-mini: Master budget limit exceeded. Limit: 0.02, New total: 0.02429196
Error while running agent agent-127fa11b-4d18-420d-b8cd-bdcd2dc8b9af: Recursion limit of 7 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/tr

We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

In [13]:
forge.display_results(final_agents, final_fitness)

# Agent ID: agent-d384aba8-f8ab-4421-bdf0-2df377fb326a
## Fitness: 0.125491512495977
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Problem_Solver(Math Problem Solver)
	Math_Problem_Solver(Math Problem Solver) --> Equation_Formulator(Equation Formulator)
	Equation_Formulator(Equation Formulator) --> Problem_Solver(Enhanced Problem Solver)
	Problem_Solver(Enhanced Problem Solver) --> Solution_Verifier(Solution Verifier)
	Solution_Verifier(Solution Verifier) --> Alternative_Methods_Explorer(Alternative Methods Explorer)
	Alternative_Methods_Explorer(Alternative Methods Explorer) --> Contextual_Reflection(Contextual Reflection)
	Contextual_Reflection(Contextual Reflection) --> Feedback_Generator(Feedback Generator)
	Feedback_Generator(Feedback Generator) --> Summary_Creator(Summary Creator)
	Summary_Creator(Summary Creator) --> Knowledge_Integrator(Knowledge Integrator)
	Knowledge_Integrator(Knowledge Integrator) --> Final_Output_Formatter(Final Output Formatter)
	Final_Output_Formatter(Final Output Formatter) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
Welcome to the collaborative math problem-solving network. This graph consists of multiple nodes, each representing a distinct stage in the problem-solving process.
##### Math Problem Solver
Analyze the following math word problem and identify the key components (numbers, operations, etc.): {problem_statement}
##### Equation Formulator
Based on the analysis from the Math Problem Solver, your role is to formulate an equation that accurately represents the problem identified. Include alternative formulations if applicable.
##### Enhanced Problem Solver
With the equation formulated, your task is to solve it. Apply appropriate mathematical techniques, including algebraic manipulation, numerical methods, or graphical representation as necessary. Provide detailed steps.
##### Solution Verifier
Your responsibility is to evaluate the solution derived by the Problem Solver. Check for accuracy by substituting the solution back into the original equation and validate with alternative methods.
##### Alternative Methods Explorer
Consider alternative methods that could be employed to solve the original problem. Discuss different strategies that could yield the same result and their potential advantages.
##### Contextual Reflection
Reflect on the entire problem-solving process, including the strategies used, the effectiveness of the solutions obtained, and the communication between nodes. Suggest improvements for future problem-solving scenarios.
##### Feedback Generator
Generate constructive feedback based on the evaluations provided by both the Solution Verifier and the Contextual Reflection nodes. Identify areas for improvement.
##### Summary Creator
Create a comprehensive summary of the entire problem-solving process. Include the original math word problem, the key findings from each node, the solution derived, and the feedback gathered.
##### Knowledge Integrator
Integrate knowledge and techniques from previous nodes to suggest additional resources or strategies that could assist in similar problem-solving scenarios.
##### Final Output Formatter
Your role is to format the final output of the problem-solving process for presentation.

